In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [2]:
channel1 = 'Scikitlearn'
channel2 = 'Angular'

In [3]:
#df_issue = pd.read_csv(channel2+'/'+channel2+'_issue_tags.csv')
df_issue = pd.read_csv(channel2+'/'+channel2+'_ispy.csv')

In [4]:
df_issue.head()

,Unnamed: 0,issue,solution,issue_stem,solution_stem,issue_tags,solution_tags
0,0,[<-CODE->],I keep an eye on this channel from time to tim...,[<-code->],"i keep an ey on thi channel from time to time,...",[],"['channel', 'chat', 'one-to-one', 'time', 'tim..."
1,1,laco0416: [<-ISSUE->],ericmartinezr: check out my hacked together cl...,laco0416: [<-issu->],ericmartinezr: check out my hack togeth class ...,[],"['base', 'class', 'config', 'routes', 'routing']"
2,2,there isn't a.just()in Rx5 [<-LINK->] note th...,Check out this [<-LINK->]. This is the documen...,there isn't a.just()in rx5 [<-link->] note th...,check out thi [<-link->]. thi is the document ...,"['difference', 'observable', 'reactive', 'rxjs']","['document', 'documentation', 'paste', 'replac..."
3,3,Shouldn'tDynamicComponentLoader#loadIntoLocati...,gionkunz: so how could i add it inside the div...,shouldn'tdynamiccomponentload#loadintolocation...,gionkunz: so how could i add it insid the div?...,['add'],"['add', 'angular', 'angular-components', 'comp..."
4,4,Will there be a material stuff for angular 2?,Currently I am using [<-LINK->] . I wonder if ...,will there be a materi stuff for angular 2?,current i am us [<-link->] . i wonder if i am ...,"['angular', 'angular-material', 'materialize']","['go', 'path', 'using']"


In [5]:
len(df_issue)

6662

In [6]:
df_selected = df_issue[df_issue["issue_tags"] != "[]"]
df_selected.reset_index(drop=True,inplace=True)

In [7]:
len(df_selected)

5166

In [8]:
df_selected.head()

,Unnamed: 0,issue,solution,issue_stem,solution_stem,issue_tags,solution_tags
0,2,there isn't a.just()in Rx5 [<-LINK->] note th...,Check out this [<-LINK->]. This is the documen...,there isn't a.just()in rx5 [<-link->] note th...,check out thi [<-link->]. thi is the document ...,"['difference', 'observable', 'reactive', 'rxjs']","['document', 'documentation', 'paste', 'replac..."
1,3,Shouldn'tDynamicComponentLoader#loadIntoLocati...,gionkunz: so how could i add it inside the div...,shouldn'tdynamiccomponentload#loadintolocation...,gionkunz: so how could i add it insid the div?...,['add'],"['add', 'angular', 'angular-components', 'comp..."
2,4,Will there be a material stuff for angular 2?,Currently I am using [<-LINK->] . I wonder if ...,will there be a materi stuff for angular 2?,current i am us [<-link->] . i wonder if i am ...,"['angular', 'angular-material', 'materialize']","['go', 'path', 'using']"
3,5,Has anyone run into a situation where ng2 inst...,justindujardin: no.. are you sure it doesn't m...,ha anyon run into a situat where ng2 instanti ...,justindujardin: no.. ar you sure it doesn't ma...,"['components', 'instantiation', 'match', 'matc...","['match', 'matching', 'selector', 'time', 'tim..."
4,6,Who hereplans on writing all their angular 2 a...,seanpar203: the biggest issue in using pure JS...,who hereplan on write all their angular 2 app ...,seanpar203: the biggest issu in us pure js at ...,"['angular', 'typescript']","['angular', 'document', 'documentation', 'get'..."


## Preprocessing

In [9]:
def str_clean(str_list):
    str_list = str_list.replace('[','')
    str_list = str_list.replace(']','')
    str_list = str_list.replace('\'','')
    
    return str_list

In [10]:
#consider tag+issue
msg = []
for i in range(len(df_selected)):
    str_tag = str_clean(df_selected.iloc[i]["issue_tags"])
    str_msg = str(df_selected.iloc[i]["issue_stem"]).replace('\n','')
    data = str_msg + " " + str_tag
    msg.append(data)

In [138]:
#only consider tag
msg = []
for i in range(len(df_selected)):
    str_tag = str_clean(df_selected.iloc[i]["issue_tags"])
    #str_msg = str(df_selected.iloc[i]["content_stem"]).replace('\n','')
    #data = str_msg + " " + str_tag
    msg.append(str_tag)

In [11]:
len(msg)

5166

In [12]:
msg[0]

"there isn't a.just()in rx5  [<-link->] note that that's differ to reactiv-extens/rxj observ.of(1) is the same  difference, observable, reactive, rxjs"

In [13]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(msg)

In [14]:
print(X.shape)

(5166, 5414)


## Training

In [15]:
kmeans = KMeans(n_clusters=500, random_state=0).fit(X)

In [16]:
kmeans.labels_

array([377,  35,  52, ...,  51, 195, 217])

In [17]:
len(kmeans.labels_)

5166

In [18]:
df_selected["label"]=list(kmeans.labels_)

<ipython-input-18-591c60bf4ac8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["label"]=list(kmeans.labels_)


In [19]:
df_selected.head()

,Unnamed: 0,issue,solution,issue_stem,solution_stem,issue_tags,solution_tags,label
0,2,there isn't a.just()in Rx5 [<-LINK->] note th...,Check out this [<-LINK->]. This is the documen...,there isn't a.just()in rx5 [<-link->] note th...,check out thi [<-link->]. thi is the document ...,"['difference', 'observable', 'reactive', 'rxjs']","['document', 'documentation', 'paste', 'replac...",377
1,3,Shouldn'tDynamicComponentLoader#loadIntoLocati...,gionkunz: so how could i add it inside the div...,shouldn'tdynamiccomponentload#loadintolocation...,gionkunz: so how could i add it insid the div?...,['add'],"['add', 'angular', 'angular-components', 'comp...",35
2,4,Will there be a material stuff for angular 2?,Currently I am using [<-LINK->] . I wonder if ...,will there be a materi stuff for angular 2?,current i am us [<-link->] . i wonder if i am ...,"['angular', 'angular-material', 'materialize']","['go', 'path', 'using']",52
3,5,Has anyone run into a situation where ng2 inst...,justindujardin: no.. are you sure it doesn't m...,ha anyon run into a situat where ng2 instanti ...,justindujardin: no.. ar you sure it doesn't ma...,"['components', 'instantiation', 'match', 'matc...","['match', 'matching', 'selector', 'time', 'tim...",114
4,6,Who hereplans on writing all their angular 2 a...,seanpar203: the biggest issue in using pure JS...,who hereplan on write all their angular 2 app ...,seanpar203: the biggest issu in us pure js at ...,"['angular', 'typescript']","['angular', 'document', 'documentation', 'get'...",313


In [20]:
df_selected.to_csv(channel2+'/result/'+channel2+'_ispy_predicted_500_ti.csv',index = None)